# Анализ данных на Python

*Алла Тамбовцева*

## Примеры статистических тестов

In [1]:
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Общие замечания:

* в большинстве тестов по умолчанию используется двусторонняя альтернатива;
* в `scipy` приняты обозначения `'less'` и `'greater'` для левосторонних альтернатив, в `statsmodels` – `'smaller'` и `'larger'`.

### Задача 1

Даны два списка. В списке `server1` сохранены отметки о том, перешли ли покупатели к оформлению заказа в текущем дизайне сайта интернет-магазина (1 – да, 0 – нет), в списке `server2` – аналогичные отметки для сайта в обновлённом дизайне. Известно, что посетители сайта в текущем и обновлённом дизайне – разные люди, то есть выборки независимы.

In [2]:
server1 = [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 
           0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
           0, 1, 0, 1, 0, 0, 0, 0, 0, 0]

server2 = [0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 
           0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 
           0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
           0, 1]

Используя шаблон кода для функции `proportions_ztest()` ниже, проверьте гипотезу $H_0: p_1 = p_2$ против левосторонней альтернативы, приняв уровень значимости равным 5%.

In [3]:
s1, s2 = sum(server1), sum(server2)
n1, n2 = len(server1), len(server2)

# доли для наглядности

print("Конверсия 1:", s1 / n1)
print("Конверсия 2:", s2 / n2)

Конверсия 1: 0.125
Конверсия 2: 0.34375


$$
H_0: p_1 = p_2
$$

$$
H_1: p_1 < p_2
$$

In [4]:
# s1 – число успехов в первой выборке
# s2 – число успехов во второй выборке
# n1, n2 – объемы выборок
# alternative: 'two-sided', 'smaller', 'larger'

print(proportions_ztest(count = [s1, s2], 
                        nobs = [n1, n2], 
                        alternative = "smaller"))

(-2.2185299186623557, 0.013259360979715378)


Результат выше – наблюдаемое значение статистики ($z_{набл} = -2.218$) и p-value. Так как p-value меньше уровня значимости 5%, нулевую гипотезу следует отвергнуть. Значит, доли не равны, вторая доля больше первой, конверсия обновлённой версии сайта значимо выше.

Можем разбить результат на части и дописать условие для сравнение с уровнем значимости:

In [5]:
stat, pvalue = proportions_ztest(count = [s1, s2], 
                                 nobs = [n1, n2], 
                                 alternative = "smaller")

alpha = float(input("Введите уровень значимости: "))
if pvalue > alpha:
    print(f"H0 не отвергается, p-value {pvalue:.3f}")
else:
    print(f"H0 отвергается, p-value {pvalue:.3f}")

Введите уровень значимости: 0.05
H0 отвергается, p-value 0.013


### Задача 2

Дан массив значений, которые представляют собой закодированную степень согласия с утверждением:

* 1 – абсолютно не согласен,
* 2 – не согласен,
* 3 – согласен,
* 4 – абсолютно согласен.

Пусть нас интересует доля согласных с утверждением – доля ответов 3 и 4.

Используя шаблон кода для функции `binomtest()` ниже, проверьте гипотезу $H_0: p = 0.4$ против двусторонней альтернативы, приняв уровень значимости равным 5%.

In [6]:
poll = np.array([3, 4, 1, 2, 3, 4, 2, 1, 3, 4,
                 1, 2, 4, 3, 1, 2, 3, 4, 3, 4])

Самый простой способ посчитать число 3 и 4 – сформулировать условие для всех элементов массива (операции на массивах векторизованы) и получить булев массив:

In [7]:
print(poll >= 3)

[ True  True False False  True  True False False  True  True False False
  True  True False False  True  True  True  True]


Так как `True` в Python эквивалентно значению 1, а `False` – значению 0, можем, как в задаче 1, вычислить сумму по такому массиву. Так как теперь мы работаем с массивами, а не со списками, можно воспользоваться методом `.sum()` вместо базовой функции `sum()`. А число элементов запросим через атрибут `.size` – для одномерных массивов это то же, что и `len()`. 

In [8]:
yes = (poll >= 3).sum()
N = poll.size

In [9]:
# yes – число ответов 3 и 4
# N – объем выборки

print(stats.binom_test(x = yes, n = N, p = 0.4))

0.10747832022841954


Итак, гипотезы:

$$
H_0: p = 0.4 
$$

$$
H_1: p \ne 0.4 
$$

P-value больше 0.05 и даже больше 0.1, нулевая гипотеза не отвергается. Долю согласных с утверждением можно считать равной 0.4.

### Задача 3

Даны массивы значений, в которых содержится количество килограммов, которые были сброшены участниками эксперимента в течение 6 недель следования диете A и диете B.

In [10]:
weight_a = np.array([3, 2.5, 4, 3.5, 4, 4,5, 3.8, 2.7, 1.9, 3.1, 3])
weight_b = np.array([-0.5, 2, 2.5, 3, 4, 4.6, 3.2, 0, 1.5, 1.4, 3.2, 2.2, 0])

Ниже приведены описательные статистики для выборок:

In [11]:
# ddof = 1
# чтобы считалось ст отклонение на основе
# несмещенной оценки дисперсии, с (n-1) в знаменателе

print("среднее:", round(weight_a.mean(), 2), 
      "стандартное отклонение:", round(weight_a.std(ddof = 1), 2))

print("среднее:", round(weight_b.mean(), 2), 
      "стандартное отклонение:", round(weight_b.std(ddof = 1), 2))

среднее: 3.38 стандартное отклонение: 0.84
среднее: 2.08 стандартное отклонение: 1.57


Используя функцию `ttest_ind()` из `stats` библиотеки `scipy`, проверьте, можно ли считать на уровне значимости 5% обе диеты одинаково эффективными в контексте снижения веса. Альтернативную гипотезу считайте односторонней, а ее направление выберите, исходя из данных.

Так как первое среднее больше второго, логично выбрать правостороннюю альтернативу, поэтому:

$$
H_0: \mu_1 = \mu_2 \text{ (средние равны)}
$$

$$
H_1: \mu_1 > \mu_2 \text{ (первое среднее больше)}
$$

In [12]:
stats.ttest_ind(weight_a, weight_b, alternative = "greater")

Ttest_indResult(statistic=2.525776086835049, pvalue=0.00944348362040337)

P-value меньше 0.05, значит, нулевую гипотезу на основе имеющихся данных следует отвергнуть. Первую диету можно считать более эффективной, если мы предполагаем, что эффективность в данном случае оценивается через средний показатель снижения веса.